In [148]:
print("EHorizon")

EHorizon


In [149]:
DATA_PATH = "../data/raw/npi-eng/npi.txt"

Data is collected from manything.org

In [151]:
#reading .txt file
with open(DATA_PATH, encoding='utf-8') as f:
    lines = f.readlines()

# View the first 5 lines
for line in lines[:5]:
    print(line.strip())

Who?	को?	CC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #11764160 (taro)
Hide.	लुकाउनुहोस्।	CC-BY 2.0 (France) Attribution: tatoeba.org #8907581 (CK) & #11764165 (taro)
Hide.	लुक।	CC-BY 2.0 (France) Attribution: tatoeba.org #8907581 (CK) & #11764168 (taro)
Stay.	बस्नुहोस्।	CC-BY 2.0 (France) Attribution: tatoeba.org #8907595 (CK) & #11764169 (taro)
Hello!	नमस्ते!	CC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #11840274 (taro)


In [152]:
#removing metadata tags from data
cleaned_pairs = []
with open(DATA_PATH, encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 2:  # Ensure at least English and Nepali
            english = parts[0].strip()
            nepali = parts[1].strip()
            cleaned_pairs.append([english, nepali])

In [153]:
cleaned_pairs[:5]

[['Who?', 'को?'],
 ['Hide.', 'लुकाउनुहोस्।'],
 ['Hide.', 'लुक।'],
 ['Stay.', 'बस्नुहोस्।'],
 ['Hello!', 'नमस्ते!']]

In [154]:
len(cleaned_pairs)

2689

In [155]:
#converting to a dataframe
import pandas as pd

df = pd.DataFrame(cleaned_pairs, columns=['English', 'Nepali'])


In [156]:
df.sample(5)

,English,Nepali
1021,I'm also from Australia.,म पनि अस्ट्रेलिया बाट हुँ ।
909,I only eat kosher food.,म कोषेर खाना बाहेक अरु खन्न।
163,I was excited.,म उत्साहित थिएँ।
330,Are you a townie?,के तपाई शहरवासी हुनुहुन्छ?
2267,Tom is probably still waiting for you.,टम सायद अझै तिम्रो लागि पर्खिरहेको छ।


In [157]:
#basic cleaning
null_values = df.isnull().sum()
duplicate_values = df.duplicated().sum()

print("Null Values:")
print(null_values)
print("\nDuplicate Values:")
print(duplicate_values)

Null Values:
English    0
Nepali     0
dtype: int64

Duplicate Values:
0


In [158]:
# !pip install contractions

In [159]:
#imports
import re
import unicodedata
import string
import contractions

In [160]:
#example of raw text cleaning
sample_text = "Hello, how are you? You're amazing! Don't go!!! "
# sample_text = "तपाई कसलाई बोलाउदै हुनुहुन्छ?"

sample_text = contractions.fix(sample_text)
print("Contractions example : ",sample_text)
sample_text = sample_text.lower()
print("Lowercase example : ", sample_text)
exclude = ''.join(ch for ch in string.punctuation if ch not in ['?', '.', '!'])
sample_text = re.sub(rf"[{re.escape(exclude)}]", "", sample_text)
print("Punctuation example : ",sample_text)
sample_text =  re.sub(r"\s+", " ", sample_text).strip()
print("Whitespace example : ", sample_text)

Contractions example :  Hello, how are you? You are amazing! Do not go!!! 
Lowercase example :  hello, how are you? you are amazing! do not go!!! 
Punctuation example :  hello how are you? you are amazing! do not go!!! 
Whitespace example :  hello how are you? you are amazing! do not go!!!


In [161]:
def clean_text(text, lowercase=True, remove_punct=True, expand_contr=True):
    text = unicodedata.normalize('NFKC', text)

    if expand_contr:
        text = contractions.fix(text)

    if lowercase:
        text = text.lower()

    if remove_punct:
        exclude = ''.join(ch for ch in string.punctuation if ch not in ['?', '.', '!'])
        text = re.sub(rf"[{re.escape(exclude)}]", "", text)

    text = re.sub(r"\s+", " ", text).strip()

    return text


In [162]:
# Apply preprocessing to English column
df['English_clean'] = df['English'].apply(
    lambda x: clean_text(x, lowercase=True, remove_punct=True, expand_contr=True)
)




In [163]:
#dropping original columns
df.drop(columns=['English'], inplace=True)

In [164]:
df.sample(5)

,Nepali,English_clean
822,म ढोकातिर दौडिए ।,i ran toward the door.
1520,प्रहरी चौकी कहाँ छ?,where is the police station?
2682,दुई देशबीचको सम्बन्ध विगतका वर्षहरूमा चिसो र ख...,the relationship between the two countries has...
1076,किन टमले यो सबै गरे?,why did tom do all this?
2198,कृपया मलाई 2:30 मा टमलाई कल गर्न सम्झाउनुहोस्।,please remind me to call tom at 230.


In [165]:
# Save source (e.g., English)
df['English_clean'].to_csv('../data/processed/eng_src_train.txt', index=False, header=False) #already done

# Save target (e.g., German)
df['Nepali'].to_csv('../data/processed/npi_trg_train.txt', index=False, header=False)

Tokenization

I am using SentencePiece Tokenizer using  Byte Pair Encoding (BPE) algorithm

In [166]:
# !pip install sentencepiece

In [129]:
import sentencepiece as spm

In [167]:
# English tokenizer
spm.SentencePieceTrainer.train(
    input='../data/processed/src_train.txt',
    model_prefix='../data/processed/spm_eng_n',
    vocab_size=4000,
    model_type='bpe',
    character_coverage=0.9995,
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    user_defined_symbols=[".", "!", "?"]
)

# Nepali tokenizer
spm.SentencePieceTrainer.train(
    input='../data/processed/npi_trg_train.txt',
    model_prefix='../data/processed/spm_npi_e',
    vocab_size=4000,
    model_type='bpe',
    character_coverage=1.0,
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    user_defined_symbols=["।", "?"]
)

In [169]:
import sentencepiece as spm

# English tokenizer (word-level)
spm.SentencePieceTrainer.train(
    input='../data/processed/src_train.txt',
    model_prefix='../data/processed/spm_eng_word',
    vocab_size=3200,
    model_type='word',
    character_coverage=1.0,        # 1.0 is okay for English too
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3
)

# Nepali tokenizer (word-level)
spm.SentencePieceTrainer.train(
    input='../data/processed/npi_trg_train.txt',
    model_prefix='../data/processed/spm_npi_word',
    vocab_size=3200,
    model_type='word',
    character_coverage=1.0,
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3
)


In [132]:
# Loading trained Tokenization models

sp_en = spm.SentencePieceProcessor()
sp_en.load('../data/processed/spm_eng_n.model')

sp_npi = spm.SentencePieceProcessor()
sp_npi.load('../data/processed/spm_npi_e.model')

sp_en_word = spm.SentencePieceProcessor()
sp_en_word.load('../data/processed/spm_eng_word.model')
sp_npi_word = spm.SentencePieceProcessor()
sp_npi_word.load('../data/processed/spm_npi_word.model')

True

In [171]:
sample_text_en = "i love korean food."
sample_text_npi = "मलाई कोरियन खाना मन पर्छ।"

# Tokenizing sample text
tokens_en = sp_en.encode(sample_text_en, out_type=str)
tokens_npi = sp_npi.encode(sample_text_npi, out_type=str)

tokens_en_word = sp_en_word.encode(sample_text_en, out_type=str)
tokens_npi_word = sp_npi_word.encode(sample_text_npi, out_type=str)

print("English Tokens:", tokens_en)
print("Nepali Tokens:", tokens_npi)

print("English Word Tokens:", tokens_en_word)
print("Nepali Word Tokens:", tokens_npi_word)

# Detokenizing sample text
# detokenized_en = sp_en.decode(tokens_en)
# detokenized_de = sp_de.decode(tokens_de)
# print("Detokenized English:", detokenized_en)
# print("Detokenized German:", detokenized_de)

English Tokens: ['▁i', '▁love', '▁k', 'ore', 'an', '▁food', '.']
Nepali Tokens: ['▁मलाई', '▁कोरियन', '▁खाना', '▁मन', '▁पर्छ', '।']
English Word Tokens: ['▁i', '▁love', '▁korean', '▁food.']
Nepali Word Tokens: ['▁मलाई', '▁कोरियन', '▁खाना', '▁मन', '▁पर्छ।']


In [172]:
# Updating the DataFrame with tokenized columns
df['src_tokens'] = df['English_clean'].apply(lambda x: sp_en.encode(x, out_type=str))
df['trg_tokens'] = df['Nepali'].apply(lambda x: sp_npi.encode(x, out_type=str))

df['src_tokens_word'] = df['English_clean'].apply(lambda x: sp_en_word.encode(x, out_type=str))
df['trg_tokens_word'] = df['Nepali'].apply(lambda x: sp_npi_word.encode(x, out_type=str))

In [173]:
df.sample(5)

,Nepali,English_clean,src_tokens,trg_tokens,src_tokens_word,trg_tokens_word
168,म स्कीइङ गर्न गएँ।,i went skiing.,"[▁i, ▁went, ▁ski, ing, .]","[▁म, ▁स्, की, इङ, ▁गर्न, ▁गएँ, ।]","[▁i, ▁went, ▁skiing.]","[▁म, ▁स्कीइङ, ▁गर्न, ▁गएँ।]"
397,तिनीहरू फ्रान्सेली मन पराउँछन्।,they like french.,"[▁they, ▁like, ▁french, .]","[▁तिनीहरू, ▁फ्रान्सेली, ▁मन, ▁पराउँछन्, ।]","[▁they, ▁like, ▁french.]","[▁तिनीहरू, ▁फ्रान्सेली, ▁मन, ▁पराउँछन्।]"
1776,युक्रेनको राजधानी कीभ हो।,the capital of ukraine is kyiv.,"[▁the, ▁capital, ▁of, ▁, u, k, r, ain, e, ▁is,...","[▁युक्रेनको, ▁राजधानी, ▁कीभ, ▁हो, ।]","[▁the, ▁capital, ▁of, ▁ukraine, ▁is, ▁kyiv.]","[▁युक्रेनको, ▁राजधानी, ▁कीभ, ▁हो।]"
1920,नतिजा सोमबार घोषणा हुनेछ।,results will be announced monday.,"[▁results, ▁will, ▁be, ▁ann, oun, ced, ▁monday...","[▁न, ति, जा, ▁सोमबार, ▁घ, ो, ष, णा, ▁हुनेछ, ।]","[▁results, ▁will, ▁be, ▁announced, ▁monday.]","[▁नतिजा, ▁सोमबार, ▁घोषणा, ▁हुनेछ।]"
605,तपाई अहिले कति वर्षको हुनुभयो?,how old are you now?,"[▁how, ▁old, ▁are, ▁you, ▁now, ?]","[▁तपाई, ▁अहिले, ▁कति, ▁वर्षको, ▁हु, नुभयो, ?]","[▁how, ▁old, ▁are, ▁you, ▁now?]","[▁तपाई, ▁अहिले, ▁कति, ▁वर्षको, ▁हुनुभयो?]"


In [174]:
#Calculating the average length of tokens in the source and target languages
avg_src_length = df['src_tokens'].apply(len).mean()
max_src_length = df['src_tokens'].apply(len).max()
min_src_length = df['src_tokens'].apply(len).min()



avg_trg_length = df['trg_tokens'].apply(len).mean()
max_trg_length = df['trg_tokens'].apply(len).max()
min_trg_length = df['trg_tokens'].apply(len).min()

print(f"Max Source Tokens Length: {max_src_length}")
print(f"Min Source Tokens Length: {min_src_length}")
print("-" * 50)
print(f"Max Target Tokens Length: {max_trg_length}")
print(f"Min Target Tokens Length: {min_trg_length}")
print("-" * 50)
print(f"Average Source Tokens Length: {avg_src_length}")
print(f"Average Target Tokens Length: {avg_trg_length}")

Max Source Tokens Length: 30
Min Source Tokens Length: 2
--------------------------------------------------
Max Target Tokens Length: 31
Min Target Tokens Length: 2
--------------------------------------------------
Average Source Tokens Length: 7.608776496838973
Average Target Tokens Length: 7.316846411305318


In [175]:
# Converting tokens to IDs
df['src_ids'] = df['English_clean'].apply(lambda s: sp_en.encode(s, out_type=int))
df['trg_ids'] = df['Nepali'].apply(lambda s: sp_npi.encode(s, out_type=int))

df['src_ids_word'] = df['English_clean'].apply(lambda s: sp_en_word.encode(s, out_type=int))
df['trg_ids_word'] = df['Nepali'].apply(lambda s: sp_npi_word.encode(s, out_type=int))



In [176]:
df.sample(5)

,Nepali,English_clean,src_tokens,trg_tokens,src_tokens_word,trg_tokens_word,src_ids,trg_ids,src_ids_word,trg_ids_word
364,मलाई तिम्रो धेरै याद आइरहेको छ।,i miss you a lot.,"[▁i, ▁miss, ▁you, ▁a, ▁lot, .]","[▁मलाई, ▁तिम्रो, ▁धेरै, ▁याद, ▁आइरहेको, ▁छ, ।]","[▁i, ▁miss, ▁you, ▁a, ▁lot.]","[▁मलाई, ▁तिम्रो, ▁धेरै, ▁याद, ▁आइरहेको, ▁छ।]","[8, 671, 21, 9, 343, 4]","[49, 240, 93, 527, 1934, 13, 4]","[4, 392, 6, 11, 552]","[6, 51, 13, 152, 633, 9]"
1327,हाम्रो सानो सहरमा स्वागत छ।,welcome to our small town.,"[▁welcome, ▁to, ▁our, ▁small, ▁town, .]","[▁हाम्रो, ▁सानो, ▁सहरमा, ▁स्व, ाग, त, ▁छ, ।]","[▁welcome, ▁to, ▁our, ▁small, ▁town.]","[▁हाम्रो, ▁सानो, ▁सहरमा, ▁स्वागत, ▁छ।]","[1876, 10, 402, 1039, 1111, 4]","[566, 884, 2546, 529, 59, 2941, 13, 4]","[1083, 7, 122, 497, 809]","[166, 289, 1309, 1, 9]"
1607,टमले मासु नखाने बताए।,tom said he does not eat meat.,"[▁tom, ▁said, ▁he, ▁does, ▁not, ▁eat, ▁meat, .]","[▁टमले, ▁मासु, ▁न, खा, ने, ▁बताए, ।]","[▁tom, ▁said, ▁he, ▁does, ▁not, ▁eat, ▁meat.]","[▁टमले, ▁मासु, ▁नखाने, ▁बताए।]","[22, 297, 46, 176, 37, 320, 1839, 4]","[52, 741, 55, 828, 35, 808, 4]","[5, 95, 18, 49, 10, 154, 1803]","[8, 223, 2268, 323]"
2284,तपाईंले आफ्नो मोजाहरू फुकाउन आवश्यक छैन।,you do not have to take off your socks.,"[▁you, ▁do, ▁not, ▁have, ▁to, ▁take, ▁off, ▁yo...","[▁तपाईंले, ▁आफ्नो, ▁म, ोज, ाहरू, ▁फु, काउन, ▁आ...","[▁you, ▁do, ▁not, ▁have, ▁to, ▁take, ▁off, ▁yo...","[▁तपाईंले, ▁आफ्नो, ▁मोजाहरू, ▁फुकाउन, ▁आवश्यक,...","[21, 35, 37, 65, 10, 323, 330, 108, 2861, 4]","[224, 188, 6, 334, 303, 717, 2155, 542, 73, 4]","[6, 12, 10, 13, 7, 98, 176, 29, 1]","[46, 32, 2851, 2505, 156, 16]"
2561,टमसँग प्राय: फ्रेन्च बोल्ने मौका हुँदैन।,tom does not often have a chance to speak french.,"[▁tom, ▁does, ▁not, ▁often, ▁have, ▁a, ▁chance...","[▁टमसँग, ▁प्राय, :, ▁फ्रेन्च, ▁बोल्ने, ▁मौका, ...","[▁tom, ▁does, ▁not, ▁often, ▁have, ▁a, ▁chance...","[▁टमसँग, ▁प्राय:, ▁फ्रेन्च, ▁बोल्ने, ▁मौका, ▁ह...","[22, 176, 37, 616, 65, 9, 1145, 10, 358, 308, 4]","[452, 564, 2989, 545, 2644, 2427, 1141, 4]","[5, 49, 10, 223, 13, 11, 805, 7, 141, 194]","[119, 1139, 201, 1176, 1224, 480]"


In [177]:
new_df = df.copy()

#drop original columns
new_df.drop(columns=['English_clean', 'Nepali'], inplace=True)

In [178]:
bos_id_en = sp_en.bos_id()   # English BOS, usually 2
eos_id_en = sp_en.eos_id()

bos_id_de = sp_npi.bos_id()   # Nepali BOS, usually 2
eos_id_de = sp_npi.eos_id()

print("BOS ID for English:", bos_id_en)
print("EOS ID for English:", eos_id_en)
print("BOS ID for Nepali:", bos_id_de)
print("EOS ID for Nepali:", eos_id_de)

BOS ID for English: 2
EOS ID for English: 3
BOS ID for Nepali: 2
EOS ID for Nepali: 3


In [179]:
# adding BOS and EOS tokens to the source and target IDs
new_df['src_ids'] = new_df['src_ids'].apply(lambda x: [bos_id_en] + x + [eos_id_en])
new_df['trg_ids'] = new_df['trg_ids'].apply(lambda x: [bos_id_de] + x + [eos_id_de])

new_df['src_ids_word'] = new_df['src_ids_word'].apply(lambda x: [bos_id_en] + x + [eos_id_en])
new_df['trg_ids_word'] = new_df['trg_ids_word'].apply(lambda x: [bos_id_de] + x + [eos_id_de])

In [180]:
new_df.sample(5)

,src_tokens,trg_tokens,src_tokens_word,trg_tokens_word,src_ids,trg_ids,src_ids_word,trg_ids_word
1763,"[▁is, ▁tom, ▁good, ▁at, ▁speaking, ▁french, ?]","[▁के, ▁टम, ▁फ्रान्सेली, ▁बोल्नमा, ▁राम्रो, ▁छ, ?]","[▁is, ▁tom, ▁good, ▁at, ▁speaking, ▁french?]","[▁के, ▁टम, ▁फ्रान्सेली, ▁बोल्नमा, ▁राम्रो, ▁छ?]","[2, 32, 22, 223, 132, 1467, 308, 6, 3]","[2, 34, 16, 474, 2729, 175, 13, 5, 3]","[2, 9, 5, 79, 41, 675, 474, 3]","[2, 7, 5, 141, 1175, 30, 29, 3]"
2227,"[▁will, ▁you, ▁please, ▁call, ▁me, ▁this, ▁eve...","[▁के, ▁तपाईं, ▁मलाई, ▁आज, ▁बेलुका, ▁फोन, ▁गर्न...","[▁will, ▁you, ▁please, ▁call, ▁me, ▁this, ▁eve...","[▁के, ▁तपाईं, ▁मलाई, ▁आज, ▁बेलुका, ▁फोन, ▁गर्न...","[2, 102, 21, 278, 434, 78, 86, 1465, 6, 3]","[2, 34, 84, 49, 265, 1915, 672, 83, 375, 5, 3]","[2, 26, 6, 102, 227, 33, 22, 2497, 3]","[2, 7, 27, 6, 72, 2663, 218, 11, 153, 3]"
2551,"[▁i, ▁do, ▁not, ▁have, ▁time, ▁to, ▁answer, ▁a...","[▁मसँग, ▁ती, ▁सबै, ▁प्रश्नहरूको, ▁जवाफ, ▁दिने,...","[▁i, ▁do, ▁not, ▁have, ▁time, ▁to, ▁answer, ▁a...","[▁मसँग, ▁ती, ▁सबै, ▁प्रश्नहरूको, ▁जवाफ, ▁दिने,...","[2, 8, 35, 37, 65, 194, 10, 725, 186, 863, 937...","[2, 398, 244, 180, 1999, 807, 1450, 340, 73, 4...","[2, 4, 12, 10, 13, 93, 7, 343, 60, 302, 538, 3]","[2, 103, 198, 79, 747, 235, 549, 105, 16, 3]"
1538,"[▁do, ▁not, ▁try, ▁to, ▁understand, ▁this, .]","[▁यसलाई, ▁बुझ, ्ने, ▁कोसिस, ▁नगर्नुहोस्, ▁, ।]","[▁do, ▁not, ▁try, ▁to, ▁understand, ▁this.]","[▁यसलाई, ▁बुझ्ने, ▁कोसिस, ▁नगर्नुहोस्, ▁।]","[2, 35, 37, 670, 10, 555, 86, 4, 3]","[2, 356, 590, 362, 1482, 1153, 2926, 4, 3]","[2, 12, 10, 252, 7, 229, 145, 3]","[2, 89, 2650, 512, 2274, 14, 3]"
142,"[▁we, ▁have, ▁room, .]","[▁हामीसँग, ▁कोठा, ▁छ, ।]","[▁we, ▁have, ▁room.]","[▁हामीसँग, ▁कोठा, ▁छ।]","[2, 61, 65, 423, 4, 3]","[2, 1014, 872, 13, 4, 3]","[2, 21, 13, 238, 3]","[2, 338, 299, 9, 3]"


In [181]:
new_df['src_len'] = new_df['src_ids'].apply(len)
new_df['trg_len'] = new_df['trg_ids'].apply(len)

new_df['src_len_word'] = new_df['src_ids_word'].apply(len)
new_df['trg_len_word'] = new_df['trg_ids_word'].apply(len)


In [182]:
new_df.sample(5)

,src_tokens,trg_tokens,src_tokens_word,trg_tokens_word,src_ids,trg_ids,src_ids_word,trg_ids_word,src_len,trg_len,src_len_word,trg_len_word
256,"[▁tom, ▁hated, ▁this, .]","[▁टमले, ▁यसलाई, ▁घृणा, ▁गर्यो, ।]","[▁tom, ▁hated, ▁this.]","[▁टमले, ▁यसलाई, ▁घृणा, ▁गर्यो।]","[2, 22, 2927, 86, 4, 3]","[2, 52, 356, 976, 886, 4, 3]","[2, 5, 1722, 145, 3]","[2, 8, 89, 305, 304, 3]",6,7,5,6
1271,"[▁i, ▁want, ▁to, ▁cont, ri, b, ute, ▁more, .]","[▁म, ▁थप, ▁योग, द, ान, ▁गर्न, ▁चाहन्छु, ।]","[▁i, ▁want, ▁to, ▁contribute, ▁more.]","[▁म, ▁थप, ▁योगदान, ▁गर्न, ▁चाहन्छु।]","[2, 8, 114, 10, 758, 105, 2991, 2766, 325, 4, 3]","[2, 6, 1674, 1737, 2949, 64, 83, 191, 4, 3]","[2, 4, 40, 7, 1, 659, 3]","[2, 4, 1060, 2870, 11, 41, 3]",11,10,7,7
906,"[▁i, ▁must, ▁find, ▁the, ▁answer, .]","[▁यसको, ▁जवाफ, ▁मैले, ▁खोज, ्नुपर्छ, ।]","[▁i, ▁must, ▁find, ▁the, ▁answer.]","[▁यसको, ▁जवाफ, ▁मैले, ▁खोज्नुपर्छ।]","[2, 8, 494, 496, 23, 725, 4, 3]","[2, 634, 807, 88, 447, 1525, 4, 3]","[2, 4, 144, 155, 8, 712, 3]","[2, 184, 235, 12, 1751, 3]",8,8,7,6
2036,"[▁i, ▁do, ▁not, ▁remember, ▁toms, ▁w, if, es, ...","[▁मलाई, ▁टमकी, ▁श्रीमतीको, ▁नाम, ▁याद, ▁छैन, ।]","[▁i, ▁do, ▁not, ▁remember, ▁toms, ▁wifes, ▁name.]","[▁मलाई, ▁टमकी, ▁श्रीमतीको, ▁नाम, ▁याद, ▁छैन।]","[2, 8, 35, 37, 706, 295, 11, 310, 53, 604, 4, 3]","[2, 49, 978, 908, 252, 527, 73, 4, 3]","[2, 4, 12, 10, 258, 90, 1, 554, 3]","[2, 6, 311, 288, 61, 152, 16, 3]",12,9,9,8
2532,"[▁tom, ▁is, ▁out, ▁playing, ▁with, ▁the, ▁neig...","[▁टम, ▁छिमेकी, ▁के, ट, ाहरु, ▁संग, ▁बाहिर, ▁खे...","[▁tom, ▁is, ▁out, ▁playing, ▁with, ▁the, ▁neig...","[▁टम, ▁छिमेकी, ▁केटाहरु, ▁संग, ▁बाहिर, ▁खेल्दै...","[2, 22, 32, 202, 1026, 126, 23, 2057, 2181, 16...","[2, 16, 2604, 34, 2943, 856, 528, 331, 429, 30...","[2, 5, 9, 84, 370, 37, 8, 1, 2711, 3]","[2, 5, 1004, 1671, 206, 81, 1744, 9, 3]",12,13,10,9


Saving the cleaned and preprocessed dataset

In [183]:
new_df.to_csv("../data/processed/cleaned_dataset_eng_npi.csv", index=False)